# Type-Aware ES|QL Binding

This notebook demonstrates the new type-aware ES|QL binding syntax in elastic-script. This feature allows you to declare variables and bind ES|QL query results directly with type validation.

## Binding Types

| Declaration | ES|QL Result | Binding |
|-------------|--------------|----------|
| `DECLARE x ARRAY FROM <esql>` | Multiple rows | All rows as array |
| `DECLARE x DOCUMENT FROM <esql>` | Single row | Row as document |
| `DECLARE x NUMBER FROM <esql>` | 1 row, 1 column | Scalar number |
| `DECLARE x STRING FROM <esql>` | 1 row, 1 column | Scalar string |
| `DECLARE x DATE FROM <esql>` | 1 row, 1 column | Scalar date |
| `DECLARE x BOOLEAN FROM <esql>` | 1 row, 1 column | Scalar boolean |

## Setup: Create Sample Data

First, let's create some sample data to query.

In [ ]:
CREATE PROCEDURE setup_sample_data()
BEGIN
    -- Create sample logs
    CALL INDEX_DOCUMENT('test-logs', {'@timestamp': '2026-01-14T10:00:00Z', 'level': 'INFO', 'service': 'api', 'message': 'Request received'});
    CALL INDEX_DOCUMENT('test-logs', {'@timestamp': '2026-01-14T10:01:00Z', 'level': 'ERROR', 'service': 'api', 'message': 'Database connection failed'});
    CALL INDEX_DOCUMENT('test-logs', {'@timestamp': '2026-01-14T10:02:00Z', 'level': 'ERROR', 'service': 'payment', 'message': 'Payment timeout'});
    CALL INDEX_DOCUMENT('test-logs', {'@timestamp': '2026-01-14T10:03:00Z', 'level': 'WARN', 'service': 'auth', 'message': 'Invalid token'});
    CALL INDEX_DOCUMENT('test-logs', {'@timestamp': '2026-01-14T10:04:00Z', 'level': 'INFO', 'service': 'api', 'message': 'Request completed'});
    
    PRINT 'Sample data created';
END PROCEDURE

CALL setup_sample_data();

## 1. ARRAY Binding - Capture Multiple Rows

Use `ARRAY` when your query returns multiple rows and you want to capture all of them.

In [ ]:
CREATE PROCEDURE demo_array_binding()
BEGIN
    -- Capture all error logs as an array
    DECLARE errors ARRAY FROM test-logs | WHERE level == 'ERROR' | KEEP service, message;
    
    PRINT 'Found ' || ARRAY_LENGTH(errors) || ' errors:';
    
    FOR error IN errors LOOP
        PRINT '  - [' || error.service || '] ' || error.message;
    END LOOP
END PROCEDURE

CALL demo_array_binding();

## 2. DOCUMENT Binding - Single Row as Document

Use `DOCUMENT` when your query returns exactly one row with multiple columns.

In [ ]:
CREATE PROCEDURE demo_document_binding()
BEGIN
    -- Get aggregated stats as a single document
    DECLARE stats DOCUMENT FROM test-logs 
        | STATS total = COUNT(*), errors = COUNT(*) WHERE level == 'ERROR', warns = COUNT(*) WHERE level == 'WARN';
    
    PRINT 'Log Statistics:';
    PRINT '  Total logs: ' || stats.total;
    PRINT '  Errors: ' || stats.errors;
    PRINT '  Warnings: ' || stats.warns;
END PROCEDURE

CALL demo_document_binding();

## 3. NUMBER Binding - Scalar Numeric Value

Use `NUMBER` when your query returns exactly one row with one numeric column.

In [ ]:
CREATE PROCEDURE demo_number_binding()
BEGIN
    -- Get a count as a number
    DECLARE error_count NUMBER FROM test-logs | WHERE level == 'ERROR' | STATS count = COUNT(*);
    
    PRINT 'Error count: ' || error_count;
    
    -- Use in conditions
    IF error_count > 0 THEN
        PRINT 'Alert: There are ' || error_count || ' errors!';
    ELSE
        PRINT 'All clear - no errors.';
    END IF
END PROCEDURE

CALL demo_number_binding();

## 4. Practical Example: Error Rate Monitoring

A complete example showing type-aware bindings in a real-world scenario. 

In [ ]:
CREATE PROCEDURE monitor_errors(threshold NUMBER)
BEGIN
    PRINT 'Monitoring error rates (threshold: ' || threshold || '%)...';
    PRINT '';
    
    -- Get total log count
    DECLARE total NUMBER FROM test-logs | STATS count = COUNT(*);
    
    -- Get error count
    DECLARE errors NUMBER FROM test-logs | WHERE level == 'ERROR' | STATS count = COUNT(*);
    
    -- Calculate error rate
    DECLARE error_rate NUMBER = (errors / total) * 100;
    
    PRINT 'Total logs: ' || total;
    PRINT 'Error logs: ' || errors;
    PRINT 'Error rate: ' || error_rate || '%';
    PRINT '';
    
    -- Check threshold
    IF error_rate > threshold THEN
        PRINT 'ALERT: Error rate exceeds threshold!';
        
        -- Get error details
        DECLARE error_details ARRAY FROM test-logs | WHERE level == 'ERROR' | KEEP service, message;
        
        PRINT 'Error breakdown:';
        FOR error IN error_details LOOP
            PRINT '  - [' || error.service || '] ' || error.message;
        END LOOP
    ELSE
        PRINT 'All systems nominal.';
    END IF
END PROCEDURE

CALL monitor_errors(30);